In [1]:
import os
import keras
import random
import pathlib
import numpy as np
import tensorflow as tf

from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

Using TensorFlow backend.


In [2]:
tf.__version__

'1.14.0'

In [2]:
'''
First experiment testing on 1 image
'''
image = img_to_array(load_img('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/few_images/Train/01jzbs.jpg'))
image = np.array(image, dtype=float)
''' 
Convert RBG 255 colors into base 2 binary for computational ease (binary)
RGB coefficients from 0-255 would be too high for our model to process (given a typical learning rate),
thus convert to range 0-1.0
'''
image = 1.0/255*image

In [3]:
# Import map images into the lightness - a/b colorspace
X = rgb2lab(image)[:,:,0]
y = rgb2lab(image)[:,:,1:]
# Using tanh activation provides -128 to 128; normalizing it to -1 to 1
y = y/128

In [4]:
X = X.reshape(1,256,256,1)
y = y.reshape(1,256,256,2)

In [5]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=(2,2)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=(2,2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [6]:
#Train the neural network
model.fit(X, y, batch_size=1, epochs=2000)

Epoch 1/2000
1/1 [==============================] - 66s 66s/step - loss: 0.4136
Epoch 2/2000
1/1 [==============================] - 0s 80ms/step - loss: 0.5261
Epoch 3/2000
1/1 [==============================] - 0s 92ms/step - loss: 0.0415
Epoch 4/2000
1/1 [==============================] - 0s 92ms/step - loss: 0.0115
Epoch 5/2000
1/1 [==============================] - 0s 84ms/step - loss: 0.0081
Epoch 6/2000
1/1 [==============================] - 0s 92ms/step - loss: 0.0069
Epoch 7/2000
1/1 [==============================] - 0s 84ms/step - loss: 0.0071
Epoch 8/2000
1/1 [==============================] - 0s 80ms/step - loss: 0.0062
Epoch 9/2000
1/1 [==============================] - 0s 84ms/step - loss: 0.0060
Epoch 10/2000
1/1 [==============================] - 0s 84ms/step - loss: 0.0057
Epoch 11/2000
1/1 [==============================] - 0s 88ms/step - loss: 0.0055
Epoch 12/2000
1/1 [==============================] - 0s 72ms/step - loss: 0.0068
Epoch 13/2000
1/1 [==================

In [7]:
# Output colorizations on its own greyscale image
output = model.predict(X)
output = output * 128
canvas = np.zeros((256, 256, 3))
canvas[:,:,0] = X[0][:,:,0]
canvas[:,:,1:] = output[0]
imsave("single_img_result.jpg", lab2rgb(canvas))
imsave("single_img_greyscale.jpg", rgb2gray(lab2rgb(canvas)))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
